In [6]:
import librosa
import numpy as np
import pyworld

sound, sr = librosa.load('../voice/seika_voice/001.wav')
sound_double = sound.astype(np.float64)

# get info
f0, ap, sp =  pyworld.wav2world(sound_double, sr)
print(f0)
print(ap)
print(sp)

[  0.         237.24675    240.57799027 ... 177.74144098 177.12354187
 177.85948945]
[[1.02628879e-06 1.02716315e-06 1.02979459e-06 ... 1.00300298e-11
  9.32810758e-12 9.10188146e-12]
 [3.49510930e-06 3.51815227e-06 3.58280842e-06 ... 3.32550804e-10
  3.33930773e-10 3.34382431e-10]
 [1.68175772e-05 1.68616116e-05 1.69943391e-05 ... 9.30339811e-11
  8.58147868e-11 8.33529191e-11]
 ...
 [6.46139297e-04 6.46566391e-04 6.47624923e-04 ... 8.52724482e-10
  9.57860349e-10 9.95331763e-10]
 [3.73309012e-04 3.73236817e-04 3.73264052e-04 ... 7.52209196e-11
  6.50758183e-11 6.16191006e-11]
 [9.91760679e-05 9.91957283e-05 9.93368542e-05 ... 9.22824814e-11
  8.10316008e-11 7.74228782e-11]]
[[1.         1.         1.         ... 1.         1.         1.        ]
 [0.001      0.00104891 0.0011002  ... 0.99753274 0.99876561 1.        ]
 [0.001      0.00104866 0.00109969 ... 0.99924205 0.99962095 1.        ]
 ...
 [0.001      0.00104942 0.00110128 ... 0.99838981 0.99919458 1.        ]
 [0.001      0.001

* f0 = 基本周波数
* ap = 非周期性
* sp = スペクトログラム

In [8]:
import librosa
import wave
import numpy as np
import pyworld
import pyaudio
import soundfile as sf

# get sound info
sound, sr = librosa.load('../voice/seika_voice/001.wav')
sound_double = sound.astype(np.float64)
f0, sp, ap =  pyworld.wav2world(sound_double, sr)

# synthesize
make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
sf.write('aaa.wav', make_sound, sr)

## DTWでパラレルデータにしなきゃいけないっぽい

In [26]:
import librosa
import numpy as np
import pyworld
import soundfile as sf
from numpy.linalg import norm
from dtw import dtw

# get seika_sound info
sound, sr = librosa.load('../voice/seika_voice/001.wav')
seika_mfcc = librosa.feature.mfcc(y = sound, sr = sr)
#seika_sound_double = sound.astype(np.float64)
#f0, sp, ap =  pyworld.wav2world(seika_sound_double, seika_sr)

# get hiroshiba_sound info
sound, sr = librosa.load('../voice/hiroshiba_voice/001.wav')
hiro_mfcc = librosa.feature.mfcc(y = sound, sr = sr)
#hiro_sound_double = sound.astype(np.float64)
#f0, sp, ap =  pyworld.wav2world(hiro_sound_double, hiro_sr)

#DTW
dist, cost, acc_cost, path = dtw(seika_mfcc.T, hiro_mfcc.T, dist = lambda x, y: norm(x - y, ord = 1)) 
seika_conv_mfcc = seika_mfcc.T[path[0]].T
hiro_conv_mfcc = hiro_mfcc.T[path[1]].T
print(seika_conv_mfcc.shape)
print(hiro_conv_mfcc.shape)

# synthesize
 #make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
 #sf.write('aaa.wav', make_sound, sr)

(20, 318)
(20, 318)


## ISTFT(逆短時間フーリエ変換)をすると音声にできる？

In [19]:
import librosa
import numpy as np
import pyworld
import soundfile as sf
from numpy.linalg import norm
from dtw import dtw

# get wavefile info
def get_wave_info(file_name):
    sound, sr = librosa.load(file_name)
    mfcc = librosa.feature.mfcc(y = sound, sr = sr)
    stft = librosa.core.stft(sound)
    return sr, mfcc, stft

# do DTW
def DTW(mfcc_1, mfcc_2):
    dist, cost, acc_cost, path = dtw(mfcc_1.T, mfcc_2.T, dist = lambda x, y: norm(x - y, ord = 1))
    return path

# DTWed stft to sound
def stft_to_sound(stft, path):
    conv_stft = stft.T[path].T
    print(conv_stft.shape)
    conv_sound = librosa.core.istft(conv_stft)
    return conv_sound

# data to wave
def make_sound(sound, sr, file_name):
    sound_double = sound.astype(np.float64)
    f0, sp, ap = pyworld.wav2world(sound_double, sr)
    make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
    sf.write(file_name, make_sound, sr)
    
seika_sr,  seika_mfcc, seika_stft = get_wave_info('../voice/seika_silence/001.wav')
hiro_sr,  hiro_mfcc, hiro_stft = get_wave_info('../voice/hiro_silence/001.wav')

path = DTW(seika_mfcc, hiro_mfcc)

seika_conv_sound = stft_to_sound(seika_stft, path[0])
hiro_conv_sound = stft_to_sound(hiro_stft, path[1])

make_sound(seika_conv_sound, seika_sr, 'seika001.wav')
make_sound(hiro_conv_sound, hiro_sr, 'hiro001.wav')

(1025, 258)
(1025, 258)


* mfccとstftを対応づけたらできました！

## 流石にGANを作ろう

## その前にデータ群を作ろう

In [102]:
import librosa
import numpy as np
import pyworld
import soundfile as sf
from numpy.linalg import norm
from dtw import dtw
import glob
import pickle

# get wavefile info
def get_wave_info(file_name):
    sound, sr = librosa.load(file_name)
    print(sound.shape)
    mfcc = librosa.feature.mfcc(y = sound, sr = sr)
    stft = librosa.core.stft(sound)
    return sr, mfcc, stft

# do DTW
def DTW(mfcc_1, mfcc_2):
    dist, cost, acc_cost, path = dtw(mfcc_1.T, mfcc_2.T, dist = lambda x, y: norm(x - y, ord = 1))
    return path

# stft DTW
def DTW_stft(stft, path):
    dtw_stft = stft.T[path].T
    return dtw_stft

# stft to sound
def stft_to_sound(stft):
    conv_sound = librosa.core.istft(stft)
    return conv_sound

# data to wave
def make_sound(sound, sr, file_name):
    sound_double = sound.astype(np.float64)
    f0, sp, ap = pyworld.wav2world(sound_double, sr)
    make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
    sf.write(file_name, make_sound, sr)
    
# seika_info
seika_mfccs = []
seika_stfts = []
voices = glob.glob('../voice/seika_silence/*')
for voice in voices:
    sr, mfcc, stft = get_wave_info(voice)
    seika_mfccs.append(mfcc)
    seika_stfts.append(stft)

# hiro_info
hiro_mfccs = []
hiro_stfts = []
voices = glob.glob('../voice/hiro_silence/*')
for voice in voices:
    sr, mfcc, stft = get_wave_info(voice)
    print(sr)
    hiro_mfccs.append(mfcc)
    hiro_stfts.append(stft)

# DTW
seika_conv_stfts = []
hiro_conv_stfts = []
for index in range(90):
    path = DTW(seika_mfccs[index], hiro_mfccs[index])
    seika_conv_stfts.append(DTW_stft(seika_stfts[index], path[0]).T)
    hiro_conv_stfts.append(DTW_stft(hiro_stfts[index], path[1]).T)

# save list
seika_stft = open('seika_stft.txt', 'wb')
pickle.dump(seika_conv_stfts, seika_stft)

hiro_stft = open('hiro_stft.txt', 'wb')
pickle.dump(hiro_conv_stfts, hiro_stft)

(134123,)
(142369,)
(104991,)
(81302,)
(97622,)
(112488,)
(127450,)
(117091,)
(93992,)
(123939,)
(117674,)
(127147,)
(57572,)
(105167,)
(120821,)
(116071,)
(61818,)
(94523,)
(133754,)
(88961,)
(63921,)
(82172,)
(75013,)
(92360,)
(178898,)
(142489,)
(104733,)
(175303,)
(147630,)
(117025,)
(122477,)
(65281,)
(124797,)
(130111,)
(108646,)
(151090,)
(99610,)
(136367,)
(125481,)
(103868,)
(125317,)
(71593,)
(105454,)
(104702,)
(93144,)
(96865,)
(88750,)
(73654,)
(115158,)
(136706,)
(95753,)
(98887,)
(121040,)
(65975,)
(112963,)
(65373,)
(81653,)
(110628,)
(117216,)
(96261,)
(76339,)
(99241,)
(99740,)
(82042,)
(60116,)
(179689,)
(105845,)
(77490,)
(105729,)
(102157,)
(96832,)
(120193,)
(122940,)
(149953,)
(81047,)
(161664,)
(137236,)
(108234,)
(103348,)
(92125,)
(72624,)
(49538,)
(126150,)
(86105,)
(147619,)
(112243,)
(107947,)
(130810,)
(130483,)
(154815,)
(147103,)
22050
(156857,)
22050
(108884,)
22050
(87731,)
22050
(107979,)
22050
(121428,)
22050
(136936,)
22050
(129041,)
22050
(110108,)

KeyboardInterrupt: 

In [ ]:
#import librosa
import numpy as np
#import pyworld
#import soundfile as sf
#from numpy.linalg import norm
#from dtw import dtw
#import glob
import pickle

from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution1D
from keras.layers.advanced_activations import LeakyReLU

NUM_EPOCH = 20

# load list
f = open("./seika_stft.txt", "rb")
seika_stft = pickle.load(f)

f = open("./hiro_stft.txt", "rb")
hiro_stft = pickle.load(f)

def stft_to_sound(stft):
    conv_sound = librosa.core.istft(stft)
    return conv_sound

def make_sound(sound, sr, file_name):
    sound_double = sound.astype(np.float64)
    f0, sp, ap = pyworld.wav2world(sound_double, sr)
    make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
    sf.write(file_name, make_sound, sr)

def generator_model():
    model = Sequential()
    model.add(Dense(3075, input_dim = 1025))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Dense(1025))
    model.add(Activation('tanh'))
    return model
    
def discriminator_model():
    model = Sequential()
    model.add(Dense(2050, input_dim = 1025))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model
    
def train():
    #d_opt = Adam(lr = 1e-5, beta_1 = 0.1)
    discriminator = discriminator_model()
    discriminator.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')
    
    discriminator.trainable = False
    generator = generator_model()
    dcgan = Sequential([generator, discriminator])
    #g_opt = Adam(lr = 2e-4, beta_1 = 0.5)
    dcgan.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')
    for epoch in range(NUM_EPOCH):
        i = 0
        for stft in hiro_stft:
            generated_stft = generator.predict(stft, verbose = 0)
        
            X = np.concatenate((seika_stft[i], generated_stft))
            y = [1] * stft.shape[0] + [0] * stft.shape[0]
            d_loss = discriminator.train_on_batch(X, y)
            
            g_loss = dcgan.train_on_batch(stft, [1] * stft.shape[0])

            i = i + 1
        
        print(generated_stft)
        print(seika_stft[89])
    return generated_stft.T
    
gen_stft = train()
gen_sound = stft_to_sound(gen_stft)
make_sound(gen_sound, 22050, "make_sound.wav")

[[ 0.04317537  0.06456657 -0.03274063 ...  0.00811344  0.05864383
  -0.00359987]
 [-0.03987442 -0.00302752  0.01255832 ... -0.04988128 -0.00139714
  -0.00214091]
 [ 0.05605299 -0.13138908  0.13624175 ... -0.01720858  0.10958698
   0.02675173]
 ...
 [ 0.10036158 -0.00844376  0.09759335 ... -0.04049283 -0.09716246
  -0.0138415 ]
 [-0.01250398  0.01423617  0.00344178 ...  0.00315128  0.04344363
  -0.05649317]
 [-0.05610152  0.00714979 -0.08066972 ...  0.01855768  0.05923738
   0.02454771]]
[[-5.8032230e-02+0.0000000e+00j  9.1493025e-02+1.5368566e-17j
  -1.3886392e-01+3.1580008e-18j ...  2.3108509e-03-6.7358682e-18j
  -1.8531748e-03+3.0547396e-17j  1.8994655e-03+0.0000000e+00j]
 [-3.6465622e-02+0.0000000e+00j -3.3388454e-02+1.9758392e-02j
   1.1144072e-01+1.3411194e-02j ... -7.9884077e-04+3.7300691e-04j
   4.7153851e-05+9.3123666e-04j  1.0038812e-03+0.0000000e+00j]
 [-3.8767073e-02+0.0000000e+00j  4.4312604e-02-2.1583382e-02j
  -4.8826884e-02-1.8174319e-02j ... -7.2080869e-04+7.5860054e-04

[[ 0.0982369  -0.02544452  0.00391308 ...  0.0454045   0.06175166
   0.00350669]
 [ 0.01785444 -0.0129068  -0.00594835 ...  0.0790697   0.04329856
   0.03481209]
 [ 0.03537514 -0.02449928 -0.01567259 ...  0.07045374 -0.05283822
  -0.02747369]
 ...
 [ 0.03829097 -0.01720443  0.01646643 ...  0.07086615  0.03352984
   0.02005919]
 [ 0.03930673 -0.00974442  0.0207778  ...  0.06957577  0.02620998
   0.01494307]
 [ 0.04814302 -0.0129206   0.0180609  ...  0.06925765  0.03224871
   0.0191204 ]]
[[-5.8032230e-02+0.0000000e+00j  9.1493025e-02+1.5368566e-17j
  -1.3886392e-01+3.1580008e-18j ...  2.3108509e-03-6.7358682e-18j
  -1.8531748e-03+3.0547396e-17j  1.8994655e-03+0.0000000e+00j]
 [-3.6465622e-02+0.0000000e+00j -3.3388454e-02+1.9758392e-02j
   1.1144072e-01+1.3411194e-02j ... -7.9884077e-04+3.7300691e-04j
   4.7153851e-05+9.3123666e-04j  1.0038812e-03+0.0000000e+00j]
 [-3.8767073e-02+0.0000000e+00j  4.4312604e-02-2.1583382e-02j
  -4.8826884e-02-1.8174319e-02j ... -7.2080869e-04+7.5860054e-04